In [1]:
# Correr para instalar, en caso de ser necesario
!pip install torch torchvision
!pip install torch-geometric

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 16.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch.nn import Linear, Dropout
from torch_geometric.datasets import Amazon

c:\Users\angi3\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Import dataset from PyTorch Geometric
dataset = Amazon(root=".", name="Computers")

Processing...
Done!


In [4]:
# Access the first data instance
data = dataset[0]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch.utils.data import random_split

# Definir el modelo GNN
class GNN(nn.Module):
    def __init__(self, in_features, hidden_units, out_features):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_features, hidden_units)
        self.conv2 = GCNConv(hidden_units, out_features)
        self.relu = nn.ReLU()
        
    def forward(self, x, edge_index):
        x = self.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Fijar la semilla
torch.manual_seed(42)

# Definir el tamaño de cada subconjunto
num_nodes = data.num_nodes
train_size = int(0.7 * num_nodes)
val_size = int(0.15 * num_nodes)
test_size = num_nodes - train_size - val_size

# Realizar la división usando random_split
indices = torch.randperm(num_nodes)
train_indices, val_indices, test_indices = torch.split(indices, [train_size, val_size, test_size])

# Crear máscaras de entrenamiento, validación y prueba
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_indices] = True

data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.val_mask[val_indices] = True

data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask[test_indices] = True

# Inicializar el modelo, el criterio de pérdida y el optimizador
gnn = GNN(dataset.num_features, hidden_units=64, out_features=dataset.num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gnn.parameters(), lr=0.01)

# Función de entrenamiento
def train(model1, data):
    model1.train()
    optimizer.zero_grad()
    out = model1(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Función de evaluación
def evaluate(model1, data, mask):
    model1.eval()
    with torch.no_grad():
        out = model1(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[mask] == data.y[mask]).sum().item()
        accuracy = correct / mask.sum().item()
    return accuracy

# Entrenar el modelo
num_epochs = 150
for epoch in range(num_epochs):
    loss = train(gnn, data)
    train_acc = evaluate(gnn, data, data.train_mask)
    val_acc = evaluate(gnn, data, data.val_mask)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Train Acc: {train_acc*100:.2f}%, Val Acc: {val_acc*100:.2f}%")

# Evaluar el modelo final en el conjunto de prueba
test_acc = evaluate(gnn, data, data.test_mask)
print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")


Epoch 1/150, Loss: 2.2084, Train Acc: 37.62%, Val Acc: 37.92%
Epoch 2/150, Loss: 6.3843, Train Acc: 15.38%, Val Acc: 15.62%
Epoch 3/150, Loss: 3.7875, Train Acc: 26.18%, Val Acc: 25.32%
Epoch 4/150, Loss: 2.4025, Train Acc: 21.15%, Val Acc: 18.96%
Epoch 5/150, Loss: 2.4838, Train Acc: 49.88%, Val Acc: 47.82%
Epoch 6/150, Loss: 2.3604, Train Acc: 47.11%, Val Acc: 46.65%
Epoch 7/150, Loss: 2.1566, Train Acc: 43.37%, Val Acc: 43.60%
Epoch 8/150, Loss: 2.0065, Train Acc: 51.18%, Val Acc: 50.29%
Epoch 9/150, Loss: 1.8564, Train Acc: 58.04%, Val Acc: 56.21%
Epoch 10/150, Loss: 1.8241, Train Acc: 51.39%, Val Acc: 50.29%
Epoch 11/150, Loss: 1.7910, Train Acc: 48.56%, Val Acc: 48.64%
Epoch 12/150, Loss: 1.7416, Train Acc: 45.04%, Val Acc: 45.93%
Epoch 13/150, Loss: 1.7010, Train Acc: 54.01%, Val Acc: 54.41%
Epoch 14/150, Loss: 1.6427, Train Acc: 56.12%, Val Acc: 55.33%
Epoch 15/150, Loss: 1.5849, Train Acc: 50.82%, Val Acc: 49.85%
Epoch 16/150, Loss: 1.5855, Train Acc: 53.95%, Val Acc: 54.22%
E